# Aggiungo descrizione pagina
Per le pagine che presentano un nodo wikidata associato aggiungo la descrizione del nodo ottenuta tramite l'apposita API, richiamando la proprietà *descriptions*.
Prendendo ad esempio la pagina relativa a David Bowie la descrizione associata è : *British singer, musician, and actor (1947-2016)*

In [ ]:
pip install dnspython

In [1]:
import requests
import time as time
from tqdm.notebook import tqdm
from requests.exceptions import ConnectionError

In [5]:
# Connect mongodb atlas
from pymongo import MongoClient
access_token = ""
client = MongoClient(access_token)
db = client.get_database('wikitrend')
pages = db.page_topic.find() # new collection for single pages

In [6]:
# adding description to page with wikidata node associated
s = requests.session()
for page in tqdm(pages, desc = "Getting description"):
  if "wikidata_id" in page.keys():
      wikidata_id = page["wikidata_id"]
      if wikidata_id != None:
        # bulding api url
        url = "https://www.wikidata.org/w/api.php?action=wbgetentities&props=descriptions&languages=en&format=json&ids="+wikidata_id
        # make api call
        i=0
        not_found=True
        while i <30 and not_found:
            try:
                r = s.get(url=url,timeout=20)
                not_found=False
            except ConnectionError:
                time.sleep(1)
                i += 1
        risultato = r.json()
        risultato = risultato["entities"]
        wikidata_id = list(risultato.keys())[0]
        try:
          description = risultato[wikidata_id]["descriptions"]["en"]["value"]
          # updating page
          db.page_topic.update_one({"_id": page["_id"]}, {"$set":{"description": description}}, upsert = False)
        except:
          pass
